In [1]:
import os
import shutil
import zipfile
from PIL import Image
from distutils.dir_util import copy_tree
from tqdm import tqdm

import pandas as pd
import tensorflow as tf
import numpy as np

In [2]:
!cp -r /content/drive/Shareddrives/Sam/Datasets/Carla ./

In [3]:
SRC = './Carla'
DEST = './CarlaDS'
IMAGE_SHAPE = (88, 200 , 3)

In [4]:
extension = ".zip"

for item in os.listdir(SRC):
    if item.endswith(extension):
        file_name = os.path.join(SRC, item)
        zip_ref = zipfile.ZipFile(file_name)
        zip_ref.extractall(SRC)
        zip_ref.close()
        os.remove(file_name)

In [5]:
folders = os.listdir(SRC)

In [6]:
images_dir = os.path.join(SRC, 'images')
os.mkdir(images_dir, 0o777)

In [10]:
csv = os.path.join(SRC, folders[0], 'data.csv')
pd.read_csv(csv)

,image_name,steer,throttle,brake,speed,x,y,z,yaw,speed_limit,is_traffic_light,traffic_light_state
0,2022-03-23_16-52-23_114.jpg,0.000000,0.000000,0.0,0.000000,-3.680001,270.729950,0.221564,-89.999817,30.0,False,2
1,2022-03-23_16-52-23_115.jpg,0.000000,0.000000,0.0,0.000000,-3.680001,270.729950,0.221564,-89.999817,30.0,False,2
2,2022-03-23_16-52-23_116.jpg,0.000000,0.000000,0.0,0.000000,-3.680001,270.729950,0.221564,-89.999817,30.0,False,2
3,2022-03-23_16-52-23_117.jpg,0.077474,0.850000,0.0,0.001634,-3.680001,270.729950,0.221467,-89.999817,30.0,False,2
4,2022-03-23_16-52-23_118.jpg,0.077474,0.850000,0.0,0.000302,-3.680001,270.729950,0.221424,-89.999817,30.0,False,2
...,...,...,...,...,...,...,...,...,...,...,...,...
989,2022-03-23_16-52-23_1103.jpg,-0.000622,0.808007,0.0,7.097985,-3.449877,204.181442,0.221513,-90.016182,30.0,False,2
990,2022-03-23_16-52-23_1104.jpg,-0.000622,0.808007,0.0,7.353311,-3.450036,203.885040,0.221497,-90.020142,30.0,False,2
991,2022-03-23_16-52-23_1105.jpg,-0.000622,0.808007,0.0,7.594528,-3.450235,203.556229,0.221479,-90.024406,30.0,False,2
992,2022-03-23_16-52-23_1106.jpg,0.019869,0.000000,0.7,7.821633,-3.447557,202.913574,0.221503,-89.882812,30.0,False,2


In [ ]:
coordinates = []
controls_labels = []

controls = []
env_controls = []
gps = []
yaws = []
image_names = []

In [ ]:
SKIP_START = 10
FUTURE_FRAMES = 10


for folder in tqdm(folders):
    images = os.path.join(SRC, folder, 'images')
    copy_tree(images, images_dir)

    csv = os.path.join(SRC, folder, 'data.csv')
    df = pd.read_csv(csv)
    # remove the beginning frames
    df = df.iloc[SKIP_START: , :]
    length = df.shape[0]

    # coordinates_ = df[['x', 'y', 'z']].to_numpy()
    coordinates_ = df[['x', 'y']].to_numpy()
    controls_ = df[['steer', 'throttle', 'brake', 'speed']].to_numpy()
    env_controls_ = df[['speed_limit', 'is_traffic_light', 'traffic_light_state']].to_numpy().astype(float)
    image_names_ = df['image_name'].to_numpy()
    yaws_ = df['yaw'].to_numpy()

    for i in range(1, length - FUTURE_FRAMES):
        controls.append(controls_[i-1])
        controls_labels.append(controls_[i])
        coordinates.append(coordinates_[i:i+FUTURE_FRAMES])
        env_controls.append(env_controls_[i])
        yaws.append(yaws_[i])
        image_names.append(image_names_[i])
        # gps.append()

    shutil.rmtree(os.path.join(SRC, folder), ignore_errors=True)

In [ ]:
controls = np.array(controls)
controls_labels = np.array(controls_labels)
coordinates = np.array(coordinates)
env_controls = np.array(env_controls)
yaws = np.array(yaws)
image_names = np.array(image_names)

In [ ]:
assert controls.shape[0] == controls_labels.shape[0] == coordinates.shape[0] == env_controls.shape[0] == yaws.shape[0] == image_names.shape[0]

In [ ]:
controls.shape, controls_labels.shape, coordinates.shape, env_controls.shape, yaws.shape, image_names.shape

In [ ]:
DATA_SIZE = controls.shape[0]
PARTS_LEN = 10_000

In [ ]:
indexes = np.arange(DATA_SIZE)
np.random.shuffle(indexes)
print(indexes.shape)
parts = np.array_split(indexes, DATA_SIZE//PARTS_LEN)

In [ ]:
parts[2].shape

In [ ]:
os.mkdir(DEST, 0o777)

for i, p in enumerate(parts):
    part_folder = os.path.join(DEST, str(i))
    os.mkdir(part_folder)

    controls_p = controls[p]
    controls_labels_p = controls_labels[p]
    coordinates_p = coordinates[p]
    env_controls_p = env_controls[p]
    yaws_p = yaws[p]

    names = image_names[p]
    images_p = np.empty((names.shape[0], *IMAGE_SHAPE), dtype=np.uint8)

    for ind, img_name in enumerate(names):
        image_src = os.path.join(SRC, 'images', img_name)
        image = Image.open(image_src)
        images_p[ind] = np.array(image, dtype=np.uint8)[:, :, 0:3]

    np.save(os.path.join(part_folder, 'controls'), controls_p)
    np.save(os.path.join(part_folder, 'controls_labels'), controls_labels_p)
    np.save(os.path.join(part_folder, 'coordinates'), coordinates_p)
    np.save(os.path.join(part_folder, 'env_controls'), env_controls_p)
    np.save(os.path.join(part_folder, 'yaws'), yaws_p)
    np.save(os.path.join(part_folder, 'images'), images_p)

In [ ]:
! du -sh ./CarlaDS

In [ ]:
generated_data = './CarlaDS/0'